<a href="https://colab.research.google.com/github/rafaeljosem/MNA-ProyectoIntegrador_EQ10/blob/main/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalación de dependencias
!pip install transformers datasets peft accelerate evaluate nltk numpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 18.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed May 15 00:26:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import Dataset
from peft import LoraConfig, get_peft_model
from accelerate import Accelerator
import evaluate
import nltk
import numpy as np
import pandas as pd
from tqdm import tqdm

%load_ext cudf.pandas

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
# Carga y preprocesamiento de datos
df = pd.read_csv('https://raw.githubusercontent.com/rafaeljosem/MNA-ProyectoIntegrador_EQ10/main/Avance-2/MexicanLaws_Clean_Compiled_PrePro_DataSet.csv')
df = df[['File Name', 'Text', 'Tokens']]  # Seleccionar solo las columnas necesarias
df = df.dropna().reset_index(drop=True)
df.columns = ['file_name', 'text', 'tokens']

In [13]:
df.head(10)

,file_name,text,tokens
0,Dec_HEEUM_280417.pdf.txtlimpio.txt,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['decreto', 'establece', 'horario', 'estaciona..."
1,11.pdf.txtlimpio.txt,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['estatuto', 'islas', 'marías', 'estatuto', 'i..."
2,79.pdf.txtlimpio.txt,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['impuesto', 'servicios', 'expresamente', 'dec..."
3,19.pdf.txtlimpio.txt,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['ley', 'amnistía', 'ley', 'amnistía', 'texto'..."
4,21.pdf.txtlimpio.txt,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['ley', 'ascensos', 'armada', 'méxico', 'ley',..."
5,24_171215.pdf.txtlimpio.txt,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['ley', 'asociaciones', 'religiosas', 'culto',..."
6,LAAT.pdf.txtlimpio.txt,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['ley', 'ayuda', 'alimentaria', 'trabajadores'..."
7,238.pdf.txtlimpio.txt,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['ley', 'capitalización', 'procampo', 'ley', '..."
8,30.pdf.txtlimpio.txt,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['ley', 'contribución', 'mejoras', 'obras', 'p..."
9,LCID_171215.pdf.txtlimpio.txt,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['ley', 'cooperación', 'internacional', 'desar..."


In [9]:
# Tokenización y formateo de datos
model_name = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
def format_prompt(text):
  return f"<s>[INST] {text}  [/INST]"

def format_completion(text):
  return f"[RESP] {text} [/RESP]"

def tokenize(examples):
  prompts = [format_prompt(text) for text in examples['text']]
  completions = [format_completion(text) for text in examples['text']]

  tokenized_prompts = tokenizer(prompts, max_length=512, truncation=True)
  tokenized_completions = tokenizer(completions, max_length=512, truncation=True)

  examples['input_ids'] = tokenized_prompts['input_ids']
  examples['labels'] = tokenized_completions['input_ids']

  return examples

In [17]:
dataset = Dataset.from_pandas(df)
dataset = dataset.map(tokenize, batched=True, remove_columns=['file_name', 'text', 'tokens'])

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 70
})


In [21]:
dataset.data

InMemoryTable
input_ids: list<item: int32>
  child 0, item: int32
labels: list<item: int64>
  child 0, item: int64
----
input_ids: [[[128000,45147,31868,65562,60,...,220,1591,12,2371,12],[128000,45147,31868,65562,60,...,264,5252,39944,15540,379],...,[128000,45147,31868,65562,60,...,9470,49591,220,18,12898],[128000,45147,31868,65562,60,...,7248,3904,278,47426,30540]]]
labels: [[[128000,58,68883,60,220,...,1591,12,2371,12,679],[128000,58,68883,60,220,...,5252,39944,15540,379,30607],...,[128000,58,68883,60,220,...,49591,220,18,12898,5034],[128000,58,68883,60,220,...,3904,278,47426,30540,481]]]

In [26]:
train_dataset = dataset.train_test_split(test_size=0.2)
train_dataset = train_dataset['train']
#val_dataset = dataset['test']

In [27]:
train_dataset.column_names

['input_ids', 'labels']

In [ ]:
# Carga del modelo base
model = AutoModelForCausalLM.from_pretrained(
  model_name,
  load_in_8bit=True,
  device_map='auto',
)

In [ ]:
# Configuración de LoRA
lora_config = LoraConfig(
  r=16,
  lora_alpha=32,
  lora_dropout=0.05,
  bias="none",
  task_type="CAUSAL_LM"
)

In [ ]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# Preparación del entrenamiento
training_args = TrainingArguments(
  output_dir='./lora-mexican-laws',
  learning_rate=3e-4,
  num_train_epochs=3,
  per_device_train_batch_size=4,
  per_device_eval_batch_size=4,
  weight_decay=0.02,
  evaluation_strategy='steps',
  eval_steps=200,
  save_strategy='steps',
  save_steps=200,
  save_total_limit=3,
  logging_steps=50,
  report_to='wandb',
)

In [ ]:
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=train_dataset,
  eval_dataset=val_dataset,
  data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

In [ ]:
# Fine-tuning
trainer.train()

In [ ]:
# Evaluación
metric = evaluate.load('rouge')

In [ ]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  result = metric.compute(predictions=decoded_preds, references=decoded_labels)
  return result

trainer.evaluate(eval_dataset=val_dataset, compute_metrics=compute_metrics)

In [ ]:
# Inferencia
def inference(prompt, max_length=512):
  print(f"Prompt: {prompt}")
  input_ids = tokenizer(format_prompt(prompt), return_tensors='pt').input_ids.cuda()
  response = model.generate(input_ids, max_length=max_length)
  print(f"Response: {tokenizer.decode(response[0], skip_special_tokens=True)}")

inference("¿Quién escribió la Constitución Mexicana?")

In [ ]:
# Empaquetado y compartición
model.save_pretrained('./lora-mexican-laws')
tokenizer.save_pretrained('./lora-mexican-laws')